# Highway classifier - Gabbar

## Step 1. Prepare problem

#### 1a. Load libraries

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('ticks')

In [3]:
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns', None)

In [4]:
path = '../downloads/highway-classifier/'

#### 1b. Load dataset

In [5]:
# Loading the training dataset.
training = pd.read_csv(path + 'training.csv')

In [6]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(training.shape))
training = training.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(training.shape))

Shape before dropping duplicates: (1631, 107)
Shape after dropping duplicates: (1631, 107)


In [7]:
# Creating a smaller sample to speed up workflow.
# training = training[:500]

In [8]:
training.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,47341487,0.0,1.92e+02,845,7390,291,2753.0,0,1,0,4,8,0,3,0,1,0,0.03,0.03,3,0.01,2.31e-03,0,2.89e+02,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.81,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,47276287,0.0,9.15e+06,92193,2135664,518,745.0,0,1,0,6,5,0,0,0,1,0,5.91,6.05,37,0.17,2.26e-01,0,1.21e+07,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.81,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,47509309,0.0,3.25e+03,1502,10167,148,691.0,0,1,0,2,5,0,0,0,1,0,0.07,0.07,2,0.07,0.00e+00,0,2.14e+03,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,32.55,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,47870896,0.0,0.00e+00,645,1617,34,35.0,0,1,0,2,5,0,0,1,0,0,0.00,0.00,1,0.00,0.00e+00,0,0.00e+00,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.64,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4,47276294,0.0,5.85e+06,92193,2135664,518,745.0,0,1,0,9,3,0,0,0,1,0,3.95,3.95,14,0.30,1.70e-01,0,7.80e+06,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.72,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## 2. Summarize data

#### 2a. Descriptive statistics

In [9]:
print('Shape: {}'.format(training.shape))

Shape: (1631, 107)


In [10]:
training.dtypes.head()

changeset_id             int64
changeset_harmful      float64
changeset_bbox_area    float64
user_changesets          int64
user_features            int64
dtype: object

In [11]:
training.describe()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
count,1.63e+03,1631.00,1.63e+03,1631.00,1.63e+03,1631.00,1630.00,1.63e+03,1631.00,1631.0,1631.00,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.00,1631.0,1.63e+03,1631.0,1.63e+03,1631.0,1631.0,1631.0,1.63e+03,1631.0,1631.00,1631.0,1631.0,1631.0,1631.0,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.0,1631.0,1631.0,1.63e+03,1631.0,1631.00,1631.0,1.63e+03,1631.0,1.63e+03,1631.00,1631.0,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1.63e+03,1631.00,1631.00,1.63e+03,1631.00,1631.0,1631.00,1631.00,1631.0,1631.00,1631.00,1631.00,1631.0,1631.00,1631.00,1631.0,1631.00,1631.00,1.63e+03,1.63e+03,1631.00,1631.00,1.63e+03,1631.0,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.0,1631.0,1631.0,1631.0,1631.00,1631.00,1631.00,1.63e+03,1631.00,1631.00,1631.00
mean,4.74e+07,0.02,3.69e+06,7243.35,3.06e+05,297.45,1067.19,1.84e-03,1.00,0.0,5.79,5.74,7.36e-03,0.27,0.05,0.93,0.02,0.62,0.78,12.52,0.06,0.03,0.02,5.86e+06,0.31,0.13,0.01,0.02,0.0,5.52e-03,0.0,6.13e-04,0.0,0.0,0.0,2.15e-04,0.0,0.11,0.0,0.0,0.0,0.0,14.16,6.99e-04,0.03,0.01,0.01,0.0,0.0,0.0,1.10e-04,0.0,0.55,0.0,7.97e-04,0.0,6.44e-04,0.03,0.0,0.35,0.04,0.02,0.06,0.04,0.02,0.08,0.07,4.29e-03,0.10,0.03,1.23e-03,0.17,0.8,0.15,0.60,0.1,0.11,0.06,0.43,0.1,0.34,0.16,0.1,0.14,0.02,1.84e-03,1.84e-03,0.01,0.04,1.23e-03,0.0,0.07,1.23e-03,0.07,0.07,0.08,0.0,0.0,0.0,0.0,0.03,0.04,0.09,4.29e-03,0.02,0.02,2.09
std,4.94e+05,0.14,4.68e+07,22321.72,8.78e+05,500.84,1179.92,4.29e-02,0.04,0.0,4.74,3.31,8.55e-02,0.87,0.22,0.26,0.15,4.54,5.44,50.34,0.40,0.28,0.30,9.24e+07,0.46,0.34,0.11,0.14,0.0,7.41e-02,0.0,2.48e-02,0.0,0.0,0.0,8.67e-03,0.0,3.13,0.0,0.0,0.0,0.0,13.99,1.63e-02,0.73,0.41,0.39,0.0,0.0,0.0,3.15e-03,0.0,5.24,0.0,3.22e-02,0.0,1.50e-02,0.51,0.0,0.48,0.20,0.14,0.25,0.20,0.14,0.28,0.25,6.54e-02,0.29,0.18,3.50e-02,0.38,0.4,0.36,0.49,0.3,0.31,0.23,0.49,0.3,0.47,0.37,0.3,0.34,0.13,4.29e-02,4.29e-02,0.10,0.20,3.50e-02,0.0,0.26,3.50e-02,0.26,0.26,0.27,0.0,0.0,0.0,0.0,0.16,0.20,0.29,6.54e-02,0.15,0.14,1.76
min,4.48e+07,0.00,0.00e+00,0.00,0.00e+00,0.00,-877.00,0.00e+00,0.00,0.0,2.00,1.00,0.00e+00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00e+00,0.00,0.00,0.00,0.00,0.0,0.00e+00,0.0,0.00e+00,0.0,0.0,0.0,0.00e+00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00e+00,0.00,0.00,0.00,0.0,0.0,0.0,0.00e+00,0.0,0.00,0.0,0.00e+00,0.0,0.00e+00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00e+00,0.00,0.00,0.00e+00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00e+00,0.00e+00,0.00,0.00,0.00e+00,0.0,0.00,0.00e+00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00e+00,0.00,0.00,1.00
25%,4.72e+07,0.00,4.34e+02,20.00,8.10e+01,4.00,6.00,0.00e+00,1.00,0.0,3.00,3.00,0.00e+00,0.00,0.00,1.

In [12]:
training.groupby('changeset_harmful').size()

changeset_harmful
0.0    1596
1.0      35
dtype: int64

In [13]:
training.corr(method='pearson').head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
changeset_id,1.00,0.03,1.14e-01,-3.95e-02,-6.58e-02,-1.04e-01,-0.10,1.01e-02,-1.01e-02,NaN,2.02e-02,0.03,-1.65e-02,1.74e-02,-0.09,0.07,5.25e-04,8.59e-02,8.57e-02,4.66e-02,6.29e-02,0.06,6.70e-03,7.46e-02,-0.13,-0.19,2.78e-02,-7.70e-02,NaN,-7.90e-02,NaN,6.85e-02,NaN,NaN,NaN,-4.77e-02,NaN,-3.27e-02,NaN,NaN,NaN,NaN,0.01,9.97e-03,9.05e-02,5.78e-02,-2.04e-02,NaN,NaN,NaN,2.54e-03,NaN,-8.79e-02,NaN,-2.25e-02,NaN,6.19e-02,-6.41e-02,NaN,0.02,-6.74e-02,-0.09,6.72e-02,-5.48e-03,-3.16e-02,2.01e-02,0.04,-8.95e-03,0.08,-0.06,-2.30e-02,-0.06,0.05,0.01,0.21,-4.25e-03,-9.72e-03,7.13e-03,5.95e-02,-4.47e-04,5.46e-02,-2.04e-02,7.98e-03,0.05,-0.03,2.79e-02,2.79e-02,-0.03,3.26e-02,1.19e-02,NaN,-8.43e-03,2.37e-02,-4.22e-03,0.02,1.35e-02,NaN,NaN,NaN,NaN,-5.33e-03,-0.02,-2.78e-03,-8.95e-03,-0.05,-2.75e-02,-4.69e-02
changeset_harmful,0.03,1.00,1.42e-01,-4.22e-02,-1.78e-02,-5.12e-02,-0.10,-6.36e-03,6.36e-03,NaN,5.39e-02,-0.04,8.63e-02,1.76e-02,0.03,-0.02,3.87e-03,1.12e-01,2.10e-01,2.53e-01,4.36e-03,0.02,1.63e-02,5.77e-02,0.15,-0.02,6.04e-02,-2.13e-02,NaN,-1.10e-02,NaN,-3.67e-03,NaN,NaN,NaN,-3.67e-03,NaN,-5.29e-03,NaN,NaN,NaN,NaN,-0.02,-6.36e-03,2.90e-01,-3.67e-03,-5.16e-03,NaN,NaN,NaN,-5.19e-03,NaN,2.54e-02,NaN,-3.67e-03,NaN,-6.36e-03,-9.03e-03,NaN,-0.03,9.88e-03,0.04,-4.71e-03,9.61e-02,9.56e-03,-2.98e-02,-0.02,5.50e-02,-0.03,0.02,-5.19e-03,0.01,-0.07,0.02,-0.09,2.12e-02,1.60e-02,8.70e-04,-5.92e-02,-7.59e-03,-5.18e-02,-3.02e-02,-5.29e-03,-0.06,0.01,-6.36e-03,-6.36e-03,0.03,-1.05e-02,-5.19e-03,NaN,6.14e-03,-5.19e-03,-8.05e-03,-0.02,-2.89e-02,NaN,NaN,NaN,NaN,-2.47e-02,-0.03,1.07e-02,5.50e-02,-0.02,1.04e-02,2.16e-02
changeset_bbox_area,0.11,0.14,1.00e+00,-3.84e-03,-5.35e-03,-3.03e-03,-0.01,-3.38e-03,3.38e-03,NaN,1.07e-01,-0.03,-6.77e-03,1.21e-03,-0.02,0.02,-1.25e-02,9.38e-01,9.06e-01,3.64e-01,5.27e-01,0.65,-6.37e-03,8.82e-01,0.05,0.03,-8.79e-03,-1.11e-02,NaN,-5.87e-03,NaN,4.56e-01,NaN,NaN,NaN,-1.95e-03,NaN,-2.77e-03,NaN,NaN,NaN,NaN,-0.06,-3.38e-03,-3.25e-03,-1.94e-03,-2.65e-03,NaN,NaN,NaN,-2.76e-03,NaN,-8.31e-03,NaN,-9.38e-04,NaN,-2.90e-03,-4.75e-03,NaN,-0.06,-1.46e-02,0.04,1.41e-02,4.52e-02,-1.08e-02,7.67e-03,-0.02,-5.18e-03,0.04,-0.01,-2.76e-03,0.02,-0.12,0.08,-0.07,1.04e-02,7.66e-03,2.86e-02,-4.37e-02,1.01e-02,-5.17e-02,-3.33e-02,1.14e-02,0.04,-0.01,-3.38e-03,-3.38e-03,0.06,-1.45e-02,-2.76e-03,NaN,-2.22e-02,-2.76e-03,-2.17e-02,-0.02,-2.29e-02,NaN,NaN,NaN,NaN,-1.30e-02,-0.02,-2.48e-02,-5.18e-03,-0.01,-4.75e-03,2.08e-03
user_changesets,-0.04,-0.04,-3.84e-03,1.00e+00,7.57e-01,3.60e-01,0.10,-1.35e-02,1.35e-02,NaN,-3.59e-02,-0.10,-9.92e-03,-4.97e-02,-0.04,0.05,-2.69e-02,7.70e-03,-7.25e-04,-2.38e-02,5.71e-02,0.02,-2.30e-02,-6.42e-03,-0.16,0.58,-3.42e-02,-8.25e-03,NaN,6.19e-03,NaN,2.51e-02,NaN,NaN,NaN,-8.04e-03,NaN,1.86e-03,NaN,NaN,NaN,NaN,-0

In [14]:
skew = training.skew()
skew.sort(ascending=False)
skew.head()

route                         40.39
amenity                       40.39
leisure                       40.39
Merkaartor                    40.39
feature_node_distance_mean    37.98
dtype: float64

#### 2b. Data visualizations

In [15]:
# Histograms.
# training.hist(figsize=(20, 20));

In [16]:
# Density plot.
# training.plot.density(figsize=(20, 50), subplots=True, sharex=False);

In [17]:
# training.plot.box(layout=(5, 4), figsize=(20, 25), subplots=True, sharex=False, sharey=False);

In [18]:
# sns.heatmap(training.corr(method='pearson'));

## 3. Prepare data

#### 3a. Data cleaning

#### 3b. Feature selection

In [19]:
non_training_attributes = ['changeset_id', 'changeset_harmful']
training_attributes = list(set(training.columns) - set(non_training_attributes))

# Drop rows with any null in training attributes.
training = training.dropna(subset=training_attributes)

X = training.drop(non_training_attributes, axis=1)
y = training['changeset_harmful']

#### 3c. Data transforms

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [21]:
# Estimate importance of all features.
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X_scaled, y)

importances = pd.DataFrame(
    list(zip(X.columns, model.feature_importances_)),
    columns=['feature', 'importance']
)
importances.sort(columns='importance', ascending=False).head()

,feature,importance
4,user_days_since_first_edit,0.06
17,feature_node_count,0.06
16,feature_line_distance,0.06
0,changeset_bbox_area,0.06
1,user_changesets,0.05


## 4. Evaluate algorithms

#### 4a. Split-out validation dataset
- We have a seperate validation dataset. Yay!

#### 4b. Spot Check Algorithms
- Running algorithms in a loop below.

#### 4c. Compare Algorithms

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

models = []
models.append(('LR', LogisticRegression()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('RFC', RandomForestClassifier()))
models.append(('GBC', GradientBoostingClassifier()))

results = []
names = []
for (name, model) in models:
    kfold = KFold(n_splits=2, random_state=7)
    result = cross_val_score(model, X_scaled, y, cv=kfold, scoring='roc_auc')
    results.append(result)
    names.append(name)
    
    print('{}: {} ({})'.format(name, round(result.mean(), 2), round(result.std(), 2)))

LR: 0.78 (0.05)
CART: 0.6 (0.01)
KNN: 0.66 (0.01)
SVM: 0.81 (0.03)
RFC: 0.67 (0.03)
GBC: 0.76 (0.02)


In [23]:
# NOTE: Temporarily turning off the graph.
# fig, ax = plt.subplots(1, 1)
# plt.boxplot(results)
# ax.set_xticklabels(names)
# ax.set_ylabel('roc_auc')
# ax.set_ylim((0, 1));

## 5. Improve Accuracy

#### 5a. Algorithm Tuning

In [24]:
harmful_count = training[training['changeset_harmful'] == True].shape[0]
not_harmful_count = training[training['changeset_harmful'] == False].shape[0]

# Making this a float to prevent truncation due to integer division.
count = 1.0 * (harmful_count + not_harmful_count)

# Calculate sample weights based on number of occourances.
sample_weight = [count if 10 else 1 for l in y]

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1,  0.5],
    'max_features': ['log2'],
    'random_state': [5],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, fit_params={'sample_weight': sample_weight})
grid.fit(X_scaled, y)

print('Best score: {}'.format(grid.best_score_))
print('Best params: {}'.format(grid.best_params_))

Best score: 0.9785276073619632
Best params: {'learning_rate': 0.05, 'random_state': 5, 'max_features': 'log2', 'n_estimators': 300, 'max_depth': 3}


In [26]:
model = grid.best_estimator_
y_model = model.predict(X_scaled)

In [27]:
training['prediction'] = model.predict(X_scaled)
training.to_csv(path + 'training-review.csv', index=False)

#### 5b. Test options and evaluation metric

In [28]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y, y_model)

matrix = pd.DataFrame(matrix, index=['Labelled good', 'Labelled harmful'], columns=['Predicted good', 'Predicted harmful'])
matrix

,Predicted good,Predicted harmful
Labelled good,1595,0
Labelled harmful,12,23


In [29]:
total = matrix.sum().sum()
matrix / total

,Predicted good,Predicted harmful
Labelled good,9.79e-01,0.00
Labelled harmful,7.36e-03,0.01


In [30]:
from sklearn.metrics import classification_report

report = classification_report(y, y_model)
print(report)

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      1595
        1.0       1.00      0.66      0.79        35

avg / total       0.99      0.99      0.99      1630



In [31]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_scaled, y, cv=kfold, scoring='roc_auc')

# An area of 0.5 represents a model that is as good as random.
# An area of 1.0 represents a model that made all predictions perfectly.
print('Scores: {} ({})'.format(round(scores.mean(), 2), round(scores.std(), 2)))

Scores: 0.83 (0.02)


#### 5c. Ensembles
- Todo

## 6. Finalize model

#### 6a. Predictions on validation dataset

In [32]:
validation = pd.read_csv(path + 'validation.csv')
validation.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,46415124,0.0,29.06,10678,444702,1062,1819.0,0,1,0,2,7,0,0,0,1,0,0.01,0.01,2,0.01,0.00,0,17.81,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,4.13,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,3
1,47668474,0.0,0.00,1239,21161,426,674.0,0,1,0,4,10,0,0,1,0,0,0.00,0.00,1,0.00,0.00,0,0.00,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.64,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,50.83,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
2,47307997,0.0,27943.33,106026,2794751,737,972.0,0,1,0,2,2,0,0,0,1,0,0.27,0.27,2,0.27,0.00,0,37047.15,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.51,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,47516500,0.0,123.41,9784,828415,1486,3441.0,0,1,0,2,6,0,0,0,1,0,0.02,0.02,2,0.02,0.00,0,76.76,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,4.13,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2
4,46340958,0.0,20539.00,23,30,1,0.0,0,1,0,7,11,0,3,0,1,0,0.56,0.59,21,0.03,0.02,0,133466.89,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,4.13,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1


In [33]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(validation.shape))
validation = validation.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(validation.shape))

# Drop rows with any null in training attributes.
validation = validation.dropna(subset=training_attributes)

Shape before dropping duplicates: (700, 107)
Shape after dropping duplicates: (700, 107)


In [34]:
validation.groupby('changeset_harmful').size()

changeset_harmful
0.0    690
1.0     10
dtype: int64

In [35]:
# TODO: Replace with a pipeline
vX = validation.drop(non_training_attributes, axis=1)
vX_scaled = scaler.transform(vX)

vy = validation['changeset_harmful']

In [36]:
vy_model = model.predict(vX_scaled)

In [37]:
validation['prediction'] = model.predict(vX_scaled)
validation.to_csv(path + 'validation-review.csv', index=False)

In [38]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(vy, vy_model)
matrix = pd.DataFrame(matrix, index=['Labelled good', 'Labelled harmful'], columns=['Predicted good', 'Predicted harmful'])
matrix

,Predicted good,Predicted harmful
Labelled good,689,1
Labelled harmful,9,1


In [39]:
from sklearn.metrics import classification_report

report = classification_report(vy, vy_model)
print(report)

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       690
        1.0       0.50      0.10      0.17        10

avg / total       0.98      0.99      0.98       700



In [40]:
scores = cross_val_score(model, vX_scaled, vy, cv=kfold, scoring='roc_auc')
print('Scores: {} ({})'.format(round(scores.mean(), 2), round(scores.std(), 2)))

Scores: 0.87 (0.06)


#### 6b. Create standalone model on entire training dataset
- Todo

#### 6c. Predictions on testing dataset

In [41]:
testing = pd.read_csv(path + 'testing.csv')
testing.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,48345326,NaN,192132.11,3486,287581,619,2744.0,0,1,0,4,3,0,0,0,1,0,0.56,0.56,22,0.03,1.95e-02,0,131409.26,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,11.37,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,48346788,NaN,137.23,11155,229879,1091,1356.0,0,1,0,2,4,0,0,0,1,0,0.01,0.01,2,0.01,0.00e+00,0,95.83,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,4.56,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,2
2,48310515,NaN,1180.01,132,1068,50,865.0,0,1,0,3,6,0,0,0,1,0,0.05,0.05,4,0.02,4.57e-03,0,797.24,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,2.70,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,48309850,NaN,142922.68,19,41,11,1866.0,0,1,0,2,6,0,0,0,1,0,0.44,0.44,2,0.44,0.00e+00,0,86795.81,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,6.20,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3
4,48437377,NaN,1389.89,46227,494734,229,370.0,0,1,0,2,1,0,0,0,1,0,0.11,0.21,8,0.03,2.80e-02,0,9370.47,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,32.55,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(testing.shape))
testing = testing.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(testing.shape))

Shape before dropping duplicates: (6700, 107)
Shape after dropping duplicates: (6700, 107)


In [43]:
testing = testing.dropna(subset=training_attributes)
print('After samples rows with null: {}'.format(testing.shape))

After samples rows with null: (6700, 107)


In [44]:
# TODO: Replace with a pipeline
tX = testing.drop(non_training_attributes, axis=1)
tX_scaled = scaler.transform(tX)
# ty = testing['changeset_harmful']

In [45]:
testing['prediction'] = model.predict(tX_scaled)

In [46]:
tharmful_count = testing[testing['prediction'] == True].shape[0]
tnot_harmful_count = testing[testing['prediction'] == False].shape[0]

print('Predicted good: {}'.format(tnot_harmful_count))
print('Predicted harmful: {}'.format(tharmful_count))

print('Percentage harmful: {}%'.format(round(100.0 * tharmful_count / testing.shape[0], 2)))

Predicted good: 6694
Predicted harmful: 6
Percentage harmful: 0.09%


In [47]:
testing.to_csv(path + 'testing-review.csv', index=False)

#### 6d. Save model for later use

In [48]:
from sklearn.externals import joblib

model_path = '../gabbar/trained/model.pkl'
joblib.dump(model, model_path)

['../gabbar/trained/model.pkl']